In [38]:
%gui wx

In [39]:
from mayavi import mlab
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import sys

In [40]:
datadir = "/media/sf_VBox_Shared/London/raw/first5/"
filesdir = os.path.join(datadir, 'datawithstates/model_4states.pkl')

In [71]:
filenames = os.listdir(filesdir)
filenames = [fn for fn in filenames if os.path.isfile(os.path.join(filesdir,fn))]
datasets = [pd.read_csv(os.path.join(filesdir,fn)) for fn in filenames]
print(len(datasets))

11


In [99]:
all_data = pd.concat(datasets)
#exampledata = all_data
exampledata = datasets[2]

In [73]:
print(exampledata.columns)

Index([u'timestamp', u'acceleration', u'EN', u'anglex', u'angley', u'anglez',
       u'roll_med_acc_x', u'roll_med_acc_y', u'roll_med_acc_z',
       u'dev_roll_med_acc_x', u'dev_roll_med_acc_y', u'dev_roll_med_acc_z',
       u'invalid', u'filename', u'Slot', u'act', u'act_label', u'start_time',
       u'subset', u'switched_pos', u'state'],
      dtype='object')


In [74]:
anglex = np.array(exampledata['anglex'])
angley = np.array(exampledata['angley'])
anglez = np.array(exampledata['anglez'])
acc = np.array(exampledata['acceleration']) + 1

In [75]:
radtodegree = 180/np.pi
theta = anglex/radtodegree
psi = angley/radtodegree
phi = anglez/radtodegree

In [76]:
x = np.sin(theta)
y = np.sin(psi)
z = np.tan(phi)*np.sqrt(x**2 + y**2)

In [77]:
a_x = x*acc
a_y = y*acc
a_z = z*acc

In [78]:
#spherical coordinates
# They don work! because there is a discontinuity in phi_s
theta_s = np.arccos(a_z/acc)
phi_s = np.arctan(a_y/a_z)

In [79]:
nmax = 10000
mlab.points3d(a_x[:nmax], a_y[:nmax], a_z[:nmax], mode='point')

## Clustering

In [100]:
nclusters = 4
X_cart = np.array((a_x, a_y, a_z)).transpose()
X_sphere = np.array((anglex, angley, anglez, acc)).transpose()
X_sphere2 = np.array((theta_s, phi_s, acc)).transpose()

In [117]:
def predict_bayesian_gmm(X, nmodels):
    n_obs = X.shape[1]
    alpha_0 = 5.0
    obs_hypparams = dict(mu_0=np.zeros(n_obs),sigma_0=np.eye(n_obs),kappa_0=0.05,nu_0=5)
    model = models.Mixture(alpha_0=alpha_0,
            components=[distributions.Gaussian(**obs_hypparams) for itr in range(nmodels)])
    model.add_data(X)
    for itr in range(100):
        model.resample_model()
    return [l.z for l in model.labels_list][0]

Cluster on cartesian coordinates

In [91]:
from sklearn.mixture import GMM
from pybasicbayes import models, distributions

In [82]:
gmm = GMM(nclusters, covariance_type='full')
prediction = gmm.fit_predict(X_cart)

In [118]:
prediction = predict_bayesian_gmm(X_cart, nclusters)

In [121]:
sphere = mlab.points3d(0, 0, 0, scale_mode='none',
                                scale_factor=2,
                                color=(0.67, 0.77, 0.93),
                                resolution=50,
                                opacity=0.2)
mlab.points3d(a_x[:nmax], a_y[:nmax], a_z[:nmax], prediction[:nmax], vmax=nclusters, mode='point')

cluster on angles + acceleration

In [84]:
gmm = GMM(nclusters, covariance_type='full')
prediction_sphere = gmm.fit_predict(X_sphere)

In [123]:
prediction_sphere = predict_bayesian_gmm(X_sphere, nclusters)

In [124]:
sphere = mlab.points3d(0, 0, 0, scale_mode='none',
                                scale_factor=2,
                                color=(0.67, 0.77, 0.93),
                                resolution=50,
                                opacity=0.2)
mlab.points3d(a_x[:nmax], a_y[:nmax], a_z[:nmax], prediction_sphere[:nmax], vmax=nclusters, mode='point')

In [86]:
gmm = GMM(nclusters, covariance_type='full')
prediction_sphere2 = gmm.fit_predict(X_sphere2)

In [125]:
prediction_sphere2 = predict_bayesian_gmm(X_sphere2, nclusters)

In [126]:
sphere = mlab.points3d(0, 0, 0, scale_mode='none',
                                scale_factor=2,
                                color=(0.67, 0.77, 0.93),
                                resolution=50,
                                opacity=0.2)
mlab.points3d(a_x[:nmax], a_y[:nmax], a_z[:nmax], prediction_sphere2[:nmax], vmax=nclusters, mode='point')